In [4]:
import requests


### Get requests

**Aquí pedimos todos los mensajes de un usuario**

In [5]:
#data = requests.get('http://localhost:8080/John Wick').json()
url='http://localhost:8080/Leia Organa'

In [6]:
requests.get(url).json()

[{'text': 'Im so angry right now!', 'userName': 'Leia Organa'},
 {'text': 'Impossible! Cant do it! Just cant! He pissed me off so much.',
  'userName': 'Leia Organa'},
 {'text': 'But how am I suposed to fix the facial recognition system if it doesnt even work?',
  'userName': 'Leia Organa'},
 {'text': 'And who the hell is Tony? \U0001f92f️', 'userName': 'Leia Organa'},
 {'text': 'Oh shit. I think youre friends with my brother.',
  'userName': 'Leia Organa'}]

**Aquí pedimos todos los mensajes de un chat**

In [6]:
url='http://localhost:8080/chat/3/list'
chat2=requests.get(url).json()
chat2

[{'text': 'I want pictures of Spiderman and I want them now!',
  'userName': 'J. Jonah Jameson'},
 {'text': 'Sure, boss.', 'userName': 'John Wick'},
 {'text': 'Youve got to take me seriously.', 'userName': 'J. Jonah Jameson'},
 {'text': 'I do! I just dont feel appreciated in my job',
  'userName': 'John Wick'},
 {'text': 'Thats why youve got to get those damm pictures!!',
  'userName': 'J. Jonah Jameson'},
 {'text': 'I dunno...', 'userName': 'John Wick'},
 {'text': 'Get the god damn motherfucking fuckity fuck bloody ballsack pictures',
  'userName': 'J. Jonah Jameson'},
 {'text': 'Maybe', 'userName': 'John Wick'},
 {'text': 'Youve got to! And I believe in you',
  'userName': 'J. Jonah Jameson'},
 {'text': 'Thank you, boss! I feel much better...', 'userName': 'John Wick'}]

**Solicitamos el informe de sentimiento de un chat.**

In [7]:
url='http://localhost:8080/chat/4/sentiment'
res=requests.get(url)
res

<Response [200]>

In [8]:
report_chat=res.json()
report_chat

{'Complete chat sentiment Metric': {'Negative': 0.14709999999999998,
  'Neutral': 0.16599999999999998,
  'Positive': 0.6869},
 'Message 1 analysis': ['Danny Torrances Finger: REDRUM',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 10 analysis': ['Danny Torrance: No! That places creeps me out..',
  {'compound': -0.3595, 'neg': 0.333, 'neu': 0.667, 'pos': 0.0}],
 'Message 2 analysis': ['Danny Torrance: What was that?',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 3 analysis': ['Danny Torrances Finger: REDRUM REDRUM REDRUM',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 4 analysis': ['Danny Torrance: Did you mean red room? Id love a red room.',
  {'compound': 0.6369, 'neg': 0.0, 'neu': 0.656, 'pos': 0.344}],
 'Message 5 analysis': ['Danny Torrances Finger: Murder, bro, cold blooded splashy murderous fun',
  {'compound': -0.765, 'neg': 0.549, 'neu': 0.247, 'pos': 0.204}],
 'Message 6 analysis': ['Danny Torrance: Chill out, p

**Solicitamos recomendación para un usuario.**

In [7]:
url='http://localhost:8080/user/John Wick/recommend'
res=requests.get(url)
res

KeyboardInterrupt: 

In [ ]:
recomendation=res.json()
recomendation

### Post requests

**Añadimos un nuevo usuario con su mensaje y todos los datos del documento a la base de datos.**

In [2]:
url='http://localhost:8080/user/create'

In [3]:
new_doc = {'userName': 'John Snow', 
            'text': "You're right."}

requests.post(url, data=new_doc)

<Response [200]>

**Añadimos contenido a un chat: un mensaje de un usuario ya existente, o un nuevo usuario y su mensaje**

In [31]:
url='http://localhost:8080/chat/6/add'

In [32]:
new_doc = {'userName': 'Skyler White', 
            'text': "Roped into working for - unable to even quit. You told me that yourself, Walt. Jesus, what was I thinking? Walt, please, let's both of us stop trying to justify this whole thing and admit you're in danger."}

requests.post(url, data=new_doc)

<Response [200]>

In [33]:
url='http://localhost:8080/chat/6/add'

In [34]:
new_doc = {'userName': 'Walter White', 
            'text': "Who are you talking to right now? Who is it you think you see? Do you know how much I make a year? I mean, even if I told you, you wouldn't believe it. Do you know what would happen if I suddenly decided to stop going into work? A business big enough that it could be listed on the NASDAQ goes belly-up - disappears. It ceases to exist without me. No. You clearly don't know who you're talking to, so let me clue you in. I am not in danger, Skyler. I am the danger. A guy opens his door and gets shot, and you think that of me? No. I am the one who knocks."}

requests.post(url, data=new_doc)


<Response [200]>